In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder.appName("CovidAnalysis").getOrCreate()

covid_df = spark.read.csv("/data/covid_data.csv", header=True, inferSchema=True)

In [16]:
#Выберите 15 стран с наибольшим процентом переболевших на 31 марта (в выходящем датасете необходимы колонки: iso_code, страна, процент переболевших)
from pyspark.sql.functions import max, lit

march_31_df = covid_df.filter(col("date") == "2021-03-31")
march_31_df = march_31_df.withColumn("perc_recovered", (col("total_cases") / col("population")) * 100)
top_15_countries = march_31_df.select("iso_code", "location", "perc_recovered") \
                               .orderBy(col("perc_recovered").desc()) \
                               .limit(15)
top_15_countries.write.mode("overwrite").csv("/home/jovyan/top_15_countries.csv", header=True)

In [17]:
#Top 10 стран с максимальным зафиксированным кол-вом новых случаев за последнюю неделю марта 2021 в отсортированном порядке по убыванию
#(в выходящем датасете необходимы колонки: число, страна, кол-во новых случаев)
last_week_march = covid_df.filter((col("date") >= "2021-03-24") & (col("date") <= "2021-03-31"))

top_10_countries = last_week_march.groupBy("iso_code", "location") \
                                  .sum("new_cases") \
                                  .withColumnRenamed("sum(new_cases)", "total_new_cases") \
                                  .orderBy(col("total_new_cases").desc()) \
                                  .limit(10)
top_10_countries.write.mode("overwrite").csv("/home/jovyan/top_10_countries.csv", header=True)

In [19]:
#Посчитайте изменение случаев относительно предыдущего дня в России за последнюю неделю марта 2021. 
#(например: в россии вчера было 9150 , сегодня 8763, итог: -387) 
#(в выходящем датасете необходимы колонки: число, кол-во новых случаев вчера, кол-во новых случаев сегодня, дельта)
russia_df = covid_df.filter((col("location") == "Russia") & (col("date") >= "2021-03-24") & (col("date") <= "2021-03-31"))

from pyspark.sql.window import Window
from pyspark.sql.functions import lag

windowSpec = Window.partitionBy("location").orderBy("date")
russia_df = russia_df.withColumn("prev_day_cases", lag("new_cases").over(windowSpec))
russia_df = russia_df.withColumn("delta", col("new_cases") - col("prev_day_cases"))
russia_df.write.mode("overwrite").csv("/home/jovyan/russia_delta_cases.csv", header=True)